In [53]:
import requests
import selenium
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup

from tqdm import tqdm

import time

import pandas as pd

In [86]:
starbucks_url = "https://www.starbucks.co.kr/store/store_map.do"
driver = webdriver.Chrome("./driver/chromedriver 2")
driver.get(starbucks_url)


sido = dict()

#지역검색
xpath_1 = '//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/header[2]' # 지역검색 탭
xpath_2 = '//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li' # 시도 리스트
xpath_3 = '//*[@id="mCSB_2_container"]/ul/li[1]/a' # 

# 페이지 로드까지 기다리기 최대 10초
driver.implicitly_wait(10)

driver.find_element(By.XPATH, xpath_1).click()

for region in tqdm(driver.find_elements(By.XPATH, xpath_2)):

    region_text = region.text
    # 시도 지역 클릭
    region.click()
    time.sleep(1)
    
    # 전체 클릭
    if region_text != '세종': # 세종은 시군구 정보가 없음
        driver.find_element(By.XPATH, xpath_3).click()    
    
    # 매장 정보 불러오기 기다리기 최대 10초
    driver.implicitly_wait(10)
    time.sleep(1)
    
    sido[region_text] = driver.page_source
    time.sleep(1)
    
    # 지역검색 클릭
    driver.find_element(By.XPATH, xpath_1).click()
    


/var/folders/jt/jw2pb8hx71d48w0rx0cgqf_80000gn/T/ipykernel_8873/3685107152.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./driver/chromedriver 2")
100%|███████████████████████████████████████████| 17/17 [00:57<00:00,  3.38s/it]


In [95]:
driver.quit()

In [92]:
# 스타벅스 데이터 프레임 만들기

starbucks = pd.DataFrame()

for key, html in sido.items():
    #  지역 html가져오기
    soup = BeautifulSoup(html, "html.parser")
    sb_stores_raw = soup.find_all("ul", class_='quickSearchResultBoxSidoGugun')
    sb_stores_list = sb_stores_raw[0].find_all("li")

    sb_store_name = []
    sb_stroe_adress = []
    sb_store_gu = []
    for sb_stores in sb_stores_list:
        sb_store_name.append(sb_stores.find("strong").text)
        adress = sb_stores.find("p").text
        sb_stroe_adress.append(adress.split(",")[0])

        adress_split = adress.split()
        sb_store_gu.append(adress_split[1])
        
    
    tmp = pd.DataFrame({
    "매장명" : sb_store_name,
    "지역구" : sb_store_gu,
    "주소" : sb_stroe_adress
    })
    
    tmp['main_region'] = key
    
    starbucks = pd.concat([starbucks, tmp])

In [93]:
len(starbucks)

1807

In [94]:
starbucks[starbucks['main_region']=='세종']

,매장명,지역구,주소,main_region
0,세종다정DT,다정북로,세종특별자치시 다정북로 219 (다정동)1522-3232,세종
1,세종보람,한누리대로,세종특별자치시 한누리대로 2165 (보람동) A118,세종
2,세종새롬,새롬중앙로,세종특별자치시 새롬중앙로 64 (새롬동)1522-3232,세종
3,세종어진R,한누리대로,세종특별자치시 한누리대로 499 (어진동)1522-3232,세종
4,세종종촌,도움3로,세종특별자치시 도움3로 105-10 (종촌동) 성운프라자1522-3232,세종
5,세종첫마을,한누리대로,세종특별자치시 한누리대로 169 행복타워 1층1522-3232,세종
6,세종이마트,금송로,세종특별자치시 금송로 687(가람동 406) 이마트 세종점1522-3232,세종
7,세종청사,도움8로,세종특별자치시 도움8로 81 (어진동)1522-3232,세종
8,세종아름,아름서1길,세종특별자치시 아름서1길 13-9 (아름동) 103,세종
9,세종한솔,노을3로,세종특별자치시 노을3로 101 (한솔동)1522-3232,세종


In [155]:
starbucks['reveiw_url'] = ''

In [183]:
from selenium.webdriver.common.keys import Keys
#URL 저장 리스트
review_url = []

#검색을 위한 기본 지도창 열기
driver = webdriver.Chrome("./driver/chromedriver 2")
url = "https://map.naver.com/v5/search/스타벅스"
driver.get(url)
search_box = driver.find_element(By.CLASS_NAME, "input_search")

# 스타벅스 매장명으로 검색하여 방문자 리뷰 링크 가져오기
for idx, store in tqdm(enumerate(starbucks['매장명'].values)):
    time.sleep(2.5)
    search_box.clear()
    store_name = "스타벅스 " + store.strip() + "점"
    search_box.send_keys(store_name)
    search_box.send_keys(Keys.ENTER)
    time.sleep(2.5)
    
    try:
        driver.switch_to.frame("entryIframe")
    except:
        driver.switch_to.frame("searchIframe")
        result_stores = driver.find_elements(By.CSS_SELECTOR,
                                    "#_pcmap_list_scroll_container > ul > li > div > a > div > div > span.place_bluelink")

		#검색 결과가 여러개일때 검색 단어와 일치하는 매장 선택
        print(result_stores)
        cnt = 0
        for name in result_stores:
            if store_name == name: break
            cnt += 1
            
        #검색 결과에 검색단어와 일치하는 매장이 없는 경우
        if len(result_stores) == (cnt):
            time.sleep(2)
            driver.switch_to.default_content()
            review_url.append(np.nan)
            continue
            
        result_stores[cnt].click()
        time.sleep(2)
        driver.switch_to.default_content()
        driver.switch_to.frame("entryIframe")

    review_url = (
        driver.find_elements(By.CLASS_NAME, "flicking-camera")[0].find_elements(By.TAG_NAME, "a")[3].get_attribute(
            "href"))
    starbucks['reveiw_url'].iloc[idx] = review_url
    
    driver.switch_to.default_content()
    

    
#     break

driver.close()

/var/folders/jt/jw2pb8hx71d48w0rx0cgqf_80000gn/T/ipykernel_8873/3647108927.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./driver/chromedriver 2")
13it [01:13,  5.45s/it]

[<selenium.webdriver.remote.webelement.WebElement (session="4d68c44495625df5672e3c8928cfd8bb", element="460024d5-5e1a-4e31-bdcc-240423cf1e86")>]


13it [01:21,  6.23s/it]


AttributeError: 'str' object has no attribute 'append'

In [173]:
driver.quit()

In [181]:
name.text

MaxRetryError: HTTPConnectionPool(host='localhost', port=61614): Max retries exceeded with url: /session/ad13575cd6856aa1e3ae85a96845bcdd/element/95278a5c-3ab3-4cc7-b6cb-792260c6b17a/text (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd84428a4f0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [184]:
driver.find_elements(By.CSS_SELECTOR,
                                    "#_pcmap_list_scroll_container > ul > li > div > a > div > div > span.place_bluelink")



[]